# Helpers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp39-cp39-linux_x86_64.whl size=255891 sha256=1fefb143baa81e48f75f4867db3bb702cd8e80eef8c8153431e8b43b23de0ef1
  Stored in directory: /root/.cache/pip/wheels/f2/7a/49/9bef8878949914ecb90c08fc5bf30a05e17f475fe7e08b63a8
Successfully built pickle5


In [ ]:
import os
import pickle5 as pickle
from sklearn.preprocessing import StandardScaler
import numpy as np
from pathlib import Path
import random
import string
import math
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data

In [ ]:
def dot_product(l1,l2):

  # function to perform element wise dot product of two list
  s = 0
  for i,j in zip(l1,l2):
    ele = i*j
    s = s + ele
  return s

def multiply_scalor(k,l1):

  # function to perform a list with a scalor value
  l2 = []
  for i in l1:
    j = i * k
    l2.append(j)
  return l2

def normalize(l1):

  # function to normalize a list using standard scaler
  data = np.array(l1).reshape(-1,1)
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(data)
  l2 = scaled_data.transpose()[0]
  return list(l2)

# Code

In [ ]:
# movie information
text_meta = load_pkl("/content/drive/MyDrive/Recommendation_Journal/latest_movies_meta_info.obj")
video = os.listdir("/content/drive/MyDrive/Recommendation_system/trailer")
genre_info = load_pkl("/content/drive/MyDrive/Recommendation_Journal/genre_info_dict.obj")

# movie embedding
video_embed = load_pkl("/content/drive/MyDrive/Recommendation_Journal/movie_embedding/genre_based_movie_video_embeddings_scene_detect.pkl")
audio_embed = load_pkl("/content/drive/MyDrive/Recommendation_Journal/movie_embedding/genre_based_movie_audio_embedding.obj")
audio_video = load_pkl("/content/drive/MyDrive/Recommendation_Journal/movie_embedding/genre_based_movie_audio+video_embeddings.obj")

# genre preference
user_dislike = load_pkl("/content/drive/MyDrive/Recommendation_Journal/flickscore_genre_preferences/users_dislike_genre_flickscore.pkl")
user_like = load_pkl("/content/drive/MyDrive/Recommendation_Journal/flickscore_genre_preferences/users_like_genre_flickscore.pkl")
user_neutral = load_pkl("/content/drive/MyDrive/Recommendation_Journal/flickscore_genre_preferences/users_neutral_genre_flickscore.pkl")

# user information
complete_enc= load_pkl("/content/drive/MyDrive/sony_IITPatna/complete_rating_user_embedding.obj")

In [ ]:
trail = []
for i in video:
  trail.append(i[0:9])

In [ ]:
complete_enc_v2 = []
for i in range(763):
  if len(complete_enc[i]) != 0: # Removing zero length items from complete_enc
    complete_enc_v2.append(complete_enc[i])

In [ ]:
not_id = []
video_embed_mul = []
for i in range(753):
  for j in complete_enc_v2[i]:
    key = j[1]
    user = j[0]
    rating = j[2]
    gen = genre_info[key]
    dislike = user_dislike.loc[user].tolist()
    like = user_like.loc[user].tolist()
    neutral = user_neutral.loc[user].tolist()
    # print("key: ",key)
    # print("opinion: ",j[2])
    like_movie_score = dot_product(dislike[1:],gen)
    dislike_movie_score = dot_product(like[1:],gen)
    neutral_movie_score = dot_product(neutral[1:],gen)
    pref_score = (like_movie_score + 0.5 * neutral_movie_score)/(like_movie_score +
                                                                 dislike_movie_score +
                                                                 neutral_movie_score )
    # print("preference score: ",pref_score)
    if (like_movie_score == 0 and dislike_movie_score == 0 and neutral_movie_score == 0) and (key in trail):
      print("key: ",key)
      print("opinion: ",j[2])
      print("genre: ",gen)
      print("like: ",like_movie_score)
      print("dislike: ",dislike_movie_score)
      print("neutral: ",neutral_movie_score)
      print("dislike vector: ",dislike)
      print("like vector: ",like)
      print("neutral vector: ",neutral)
      print("preference score: ",pref_score)
      not_id.append(key)
      print("=================================================================================")
    j.append(gen)
    if math.isnan(pref_score):
      j.append(0.01)
      print("running")
      print("key2: ",key)
      video_embed_mul.append([user,key,0.01,rating])
    else:
      j.append(pref_score)
      video_embed_mul.append([user,key,pref_score,rating])

key:  tt0319165
opinion:  ['-1']
genre:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
like:  0.0
dislike:  0.0
neutral:  0.0
dislike vector:  ['mukesh', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
like vector:  ['mukesh', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
neutral vector:  ['mukesh', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
preference score:  nan
running
key2:  tt0319165


In [ ]:
finalop = []
for j in complete_enc_v2:
  kl = []
  id = j[0][0]
  for i in j:
     kl.append([i[1],i[4]])
    #  kl.append(i[4])
  p = {id : kl}          # Making list of dictionaries of movie_id liked (Rating = 1) by the user 
  finalop.append(p)      # Format: [{'user_id':[movie_id]}]

In [ ]:
user_embed = []
for j in range(len(finalop)):
  id  = list(finalop[j].keys())[0]
  k = finalop[j].get(id)
  lp = []
  for i in k:
     movie_id = i[0]
     pref_score = i[1]
     if movie_id in audio_video.keys():
        pref_score = 100 * pref_score
        lp.append(normalize(multiply_scalor(pref_score,audio_video.get(movie_id))))
  lp = np.array(lp) 
                                         # Making list of dictionaries of movie_id liked (Rating = 1) by the user by taking the average      
                                         # Format: [{'user_id':[textual embedding of movie_ids]}]
  pop = np.average(lp, axis=0)
  mk = {id:pop}
  user_embed.append(mk)

In [ ]:
dict_user_embed = dict()
for i in range(len(user_embed)):
  bnb = list(user_embed[i].keys())[0]
  value = user_embed[i][bnb]               
  dict_user_embed[bnb] = value    # Making dictionary of movie_id liked (Rating = 1) by the user  
                                  # Format: {'user_id':[textual embedding of movie_ids]}

# File Saving

In [ ]:
for i in l:
  if dict_user_embed[i].size == 1:  # removing users whose embeddings is nan
    print(i)
    dict_user_embed.pop(i)

In [ ]:
import pickle 
file_pi = open('/content/drive/MyDrive/Recommendation_Journal/Preference Score based User Embeddings/pref_score_based_user_audio_video_embedding.obj', 'wb') 
pickle.dump(dict_user_embed, file_pi)

In [ ]:
new_user = load_pkl("/content/drive/MyDrive/Recommendation_Journal/Preference Score based User Embeddings/pref_score_based_user_audio_video_embedding.obj")

In [ ]:
new_user['Avdesh Kumar']

array([ 0.08084971, -0.56518928,  1.0816766 , ...,  0.85524615,
       -0.06876155, -0.0425514 ])